In [13]:
from random import seed
# fixar a semente para que os resultados sejam repetíveis
seed(42)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor

## 1. Faça a análise descritiva dos dados e o tratamento dos dados.

In [4]:
# Carregando o conjunto de dados copacabana delimitrando por `;`
copacabana = pd.read_csv('copacabana.csv', delimiter=';')

In [5]:
# Retornando as colunas do dataframe com as primeiras linhas de dados
copacabana.head()

,Posicao,Quartos,Vagas,DistIpanema,DistPraia,DistFavela,RendaMedia,RendaMovel,RendaMovelRua,Vu2009,Mes,Idade,Tipologia,AreaConstruida,VAL_UNIT,X,Y
0,1,3.0,0.01,1144,311,146,969501,1028834,999168,1750,509,37.0,1,95,4379,685365.0700,7457802.680
1,0,2.0,0.01,2456,502,254,1472861,1137759,1305310,2300,484,30.0,1,71,6479,685941.5500,7459001.320
2,0,2.0,0.01,2448,772,229,1803724,1512475,1658100,2350,920,44.0,1,58,12414,685627.3900,7459080.520
3,0,2.0,0.01,1615,428,310,1124331,1370600,1247466,2200,930,43.0,1,88,11250,685438.2001,7458268.280
4,0,2.0,1.00,2358,586,287,1165764,1177933,1171849,2150,918,42.0,1,68,13382,685764.3840,7458954.513


# verificando se existem dados incoerentes com os demais

In [6]:
clean_copacabana = copacabana.dropna()
print('Qtde de linhas com anotação nula: %d' %(copacabana.shape[0]-clean_copacabana.shape[0]))

Qtde de linhas com anotação nula: 0


In [7]:
# Tipo de dados
copacabana.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1675 entries, 0 to 1674
Data columns (total 17 columns):
Posicao           1675 non-null int64
Quartos           1675 non-null float64
Vagas             1675 non-null float64
DistIpanema       1675 non-null int64
DistPraia         1675 non-null int64
DistFavela        1675 non-null int64
RendaMedia        1675 non-null int64
RendaMovel        1675 non-null int64
RendaMovelRua     1675 non-null int64
Vu2009            1675 non-null int64
Mes               1675 non-null int64
Idade             1675 non-null float64
Tipologia         1675 non-null int64
AreaConstruida    1675 non-null int64
VAL_UNIT          1675 non-null int64
X                 1675 non-null float64
Y                 1675 non-null float64
dtypes: float64(5), int64(12)
memory usage: 222.5 KB


In [ ]:
# Transformação em categóricos e númericos
copacabana[['DistIpanema','DistPraia','DistFavela','RendaMedia','RendaMovel','RendaMovelRua','Vu2009','AreaConstruida','VAL_UNIT']]=copacabana[['DistIpanema','DistPraia','DistFavela','RendaMedia','RendaMovel','RendaMovelRua','Vu2009','AreaConstruida','VAL_UNIT']].astype(np.float64) 
copacabana[['Quartos','Tipologia','Vagas','Mes','Posicao']]=copacabana[['Quartos','Tipologia','Vagas','Mes','Posicao']].astype('category') 
copacabana.info()

In [8]:
# Dados númericos (note que os dados categoricos não aparecem)
copacabana.describe()

,Posicao,Quartos,Vagas,DistIpanema,DistPraia,DistFavela,RendaMedia,RendaMovel,RendaMovelRua,Vu2009,Mes,Idade,Tipologia,AreaConstruida,VAL_UNIT,X,Y
count,1675.000000,1675.000000,1675.000000,1675.000000,1675.000000,1675.000000,1.675000e+03,1.675000e+03,1.675000e+03,1675.000000,1675.000000,1675.000000,1675.000000,1675.000000,1675.000000,1675.000000,1.675000e+03
mean,0.601791,1.767510,0.330842,2112.444179,353.662687,405.623284,1.232577e+06,1.214159e+06,1.223369e+06,2249.367164,646.903881,50.036442,1.071642,78.880597,8084.660896,685980.624667,7.458579e+06
std,0.489675,1.142523,0.576720,909.254397,241.712897,215.175421,2.454474e+05,3.502902e+05,2.718984e+05,518.861409,204.075055,11.611873,0.345134,59.504466,3742.027190,630.889668,7.564990e+02
min,0.000000,0.010000,0.010000,337.000000,31.000000,15.000000,4.982560e+05,4.426680e+05,4.982560e+05,780.000000,360.000000,0.010000,1.000000,15.000000,2037.000000,685024.810000,7.456842e+06
25%,0.000000,1.000000,0.010000,1414.000000,179.000000,226.500000,1.083455e+06,1.000877e+06,1.044084e+06,2000.000000,445.000000,44.000000,1.000000,38.000000,5000.000000,685472.150000,7.458048e+06
50%,1.000000,2.000000,0.010000,2259.000000,318.000000,403.000000,1.165764e+06,1.175438e+06,1.187842e+06,2200.000000,567.000000,53.000000,1.000000,63.000000,7212.000000,685792.470000,7.458776e+06
75%,1.000000,3.000000,1.000000,2805.500000,462.500000,576.500000,1.395924e+06,1.398392e+06,1.360872e+06,2500.000000,862.000000,57.000000,1.000000,98.500000,10606.000000,686490.585000,7.459228e+06
max,1.000000,6.000000,4.000000,3611.000000,1242.000000,851.000000,2.068724e+06,2.922751e+06,2.223676e+06,4100.000000,940.000000,75.000000,3.000000,668.000000,24490.000000,687253.719900,7.459698e+06


In [ ]:
# Note que as variaveis Quartos e Vagas são as que mais variam para o valor unitario
plt.figure(figsize=(20, 16))
ax=plt.subplot(221)
sns.barplot(x='VAL_UNIT', y='Quartos', data=copacabana, orient=1)
plt.title('Quartos')
ax=plt.subplot(222)
sns.barplot(x='VAL_UNIT', y='Posicao', data=copacabana, orient=1)
plt.title('Posicao')
ax=plt.subplot(223)
sns.barplot(x='VAL_UNIT', y='Vagas', data=copacabana, orient=1)
plt.title('Vagas')
ax=plt.subplot(224)
sns.barplot(x='VAL_UNIT', y='Tipologia', data=copacabana, orient=1)
plt.title('Tipologia')
plt.show()

In [ ]:
# Correlação entre os atributos (isso pode demorar, pois existem muitos atributos)
sns.pairplot(copacabana)
plt.show()

In [ ]:
# Valor Unitário por area construida e por distancia de Ipanema
f, ax = plt.subplots(figsize=(15, 15))
ax1 = sns.scatterplot(x="VAL_UNIT", y="AreaConstruida",
                hue="DistIpanema", size='RendaMovel',
                linewidth=0,
                data=copacabana, ax=ax)

In [ ]:
# Valor Unitário por area construida e por distancia da Favela
f, ax = plt.subplots(figsize=(15, 15))
ax1 = sns.scatterplot(x="VAL_UNIT", y="AreaConstruida",
                hue="DistFavela", size='RendaMovel',
                linewidth=0,
                data=copacabana, ax=ax)

In [ ]:
# Valor Unitário por area construida e por distancia da Praia
f, ax = plt.subplots(figsize=(15, 15))
ax1 = sns.scatterplot(x="VAL_UNIT", y="AreaConstruida",
                hue="DistPraia", size='RendaMovel',
                linewidth=0,
                data=copacabana, ax=ax)

## 2. Separe a feature target em um conjunto de dados separado.

In [ ]:
# Separar a variavel alvo (target) do conjunto de dados
Y = copacabana['VAL_UNIT']
X = copacabana.drop('VAL_UNIT', axis=1)
feature_names = X.columns

print(X.shape, Y.shape)

## 3. Separe agora os conjuntos em treino, validação e teste usando a proporção 70/20/10.

In [ ]:
# Separar aleatoriamente 70% para treino, 20% para validacao e 10% para teste
X_train, X_temp, Y_train, Y_temp = train_test_split(X,Y,train_size=0.7,random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp,Y_temp,train_size=0.67,random_state=42)
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape)

## 4. Normalize os dados

In [ ]:
# Analise da grandeza dos dados
plt.figure(figsize=(15, 5))
sns.boxplot(data=X_train)
plt.show()

In [ ]:
X_train.head()

In [ ]:
# transformar em numpy.array
x_train = X_train.to_numpy()
y_train = Y_train.to_numpy()
x_val = X_val.to_numpy()
y_val = Y_val.to_numpy()
x_test = X_test.to_numpy()
y_test = Y_test.to_numpy()

In [ ]:
# índice dos atributos categóricos e numéricos
categ_atrib = [0,1,2,10,12]
num_atrib = [3,4,5,6,7,8,9,11,13,14,15]
print('numéricas', num_atrib)
print('categóricas', categ_atrib)


#usar drop first para evitar colinearidade na regressão linear
#one-hot encoding somente nas colunas categóricas e normalização z-norm nas numéricas
# onehotencoder transforma uma coluna em uma coluna de vetores binarios
enc = ColumnTransformer([('onehot',OneHotEncoder(sparse=False, handle_unknown='ignore',categories='auto'), 
                          categ_atrib),
                         ('z-norm',StandardScaler(), num_atrib)], verbose=True)

print(enc)
# nao pode fazer fit em dados de teste e validacao ********************
x_train = enc.fit_transform(x_train)
x_val = enc.transform(x_val)
x_test = enc.transform(x_test)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

In [ ]:
# Os dados foram transformados para a mesma grandeza
pd.DataFrame(x_train).describe()

In [ ]:
plt.figure(figsize=(15, 5))
sns.boxplot(data=x_train)
plt.show()

## 5. Execute uma regressão linear com todas as features.

In [ ]:
# Criar o objeto de regressão linear
regr = linear_model.LinearRegression()

# Treinar o modelo usando os conjuntos de treino com as features e o target
regr.fit(x_train, y_train)

## 6. Faça a predição sobre o conjunto de testes.

In [ ]:
# Fazer as predições usando o conjunto de validação
yb_pred = regr.predict(x_val)

## 7. Calcule o Erro Médio Quadrático MAE e o Coeficiente de Determinação R2 
## Explique cada um.

In [ ]:
# Mostra os coeficientes
print('Coeficientes: \n', regr.coef_)
# Mostra o erro quadrado médio
print("Mean squared error: %.4f"
      % mean_squared_error(y_val, yb_pred))
# Coeficiente de Determinação Rˆ2 - r ao quadrado: 1 é a previsão perfeita  yval:valor real ybpred valor preditor
print('Coeficiente de Determinação: %.4f' % r2_score(y_val, yb_pred))

### Explicação: 

O Mean Squared Error (Erro Quadrático Médio) é a diferença entre o estimador e o valor ao quadrado. O coeficiente de determinação descreve o quão adequado o modelo de regressão linear está em relação aos valores analisados, o valor deve estar entre 0 e 1 onde mais aproximado de 1 é adequado.

## 8. Plote as predições em validação.

In [ ]:
# Plota a saída
plt.figure(figsize=(20, 15))
x = list(range(len(x_val)))
plt.scatter(x, y_val,  color='black')
plt.plot(x, yb_pred, color='yellow', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
# teste modelo
regrp_pred_teste = regr.predict(x_test)

# Mostra o erro quadrado médio
print("Mean squared error: %.4f"
      % mean_squared_error(y_test, regrp_pred_teste))
# Coeficiente de Determinação Rˆ2 - r ao quadrado: 1 é a previsão perfeita
print('Coeficiente de Determinação: %.4f' % r2_score(y_test, regrp_pred_teste))

In [ ]:
# Plota a saída
plt.figure(figsize=(20, 15))
x = list(range(len(x_test)))
plt.scatter(x, y_test,  color='black')
plt.plot(x, regrp_pred_teste, color='grey', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

## 9. Analise novamente os dados e remova as features que você acha que não estão agregando muita informação ao modelo. Faça uma análise de correlações para ajudar nas remoções.

In [ ]:
# Remover Quartos, Tipologia, Vagas, Posicao
x_train = X_train.drop(['Quartos','Tipologia', 'Vagas','Posicao'], axis=1).to_numpy()
x_val = X_val.drop(['Quartos','Tipologia', 'Vagas','Posicao'], axis=1).to_numpy()
x_test = X_test.drop(['Quartos','Tipologia', 'Vagas','Posicao'], axis=1).to_numpy()
print(x_train.shape, y_train.shape)

In [ ]:
plt.figure(figsize=(15, 5))
sns.boxplot(data=x_train)
plt.show()

## 10. Execute novamente os passos de 4 a 8, agora com o conjunto de dados reduzidos.

In [ ]:
# índice dos atributos categóricos e numéricos
categ_atrib = []
num_atrib = [0,1,2,3,4,5,6,7,8,9,10]
print('numéricas', num_atrib)
print('categóricas', categ_atrib)

#fazer one-hot encoding da feature bairro; 
#usar drop first para evitar colinearidade na regressão linear
#one-hot encoding somente nas colunas categóricas e normalização z-norm nas numéricas
enc = ColumnTransformer([('onehot',OneHotEncoder(sparse=False, handle_unknown='ignore'), categ_atrib),
                         ('z-norm',StandardScaler(), num_atrib)], verbose=True)

print(enc)
x_train = enc.fit_transform(x_train)
x_val = enc.transform(x_val)
x_test = enc.transform(x_test)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

In [ ]:
pd.DataFrame(x_train).describe()

In [ ]:
plt.figure(figsize=(15, 5))
sns.boxplot(data=x_train)
plt.show()

In [ ]:
# Criar o objeto de regressão linear
regrp = linear_model.LinearRegression()

# Treinar o modelo usando os conjuntos de treino com as features e o target
regrp.fit(x_train, y_train)

# Fazer as predições usando o conjunto de validaçao
yp_pred = regrp.predict(x_val)

In [ ]:
# Mostra os coeficientes
print('Coeficientes: \n', regrp.coef_)
# Mostra o erro quadrado médio
print("Mean squared error: %.4f"
      % mean_squared_error(y_val, yp_pred))
# Coeficiente de Determinação Rˆ2 - r ao quadrado: 1 é a previsão perfeita
print('Coeficiente de Determinação: %.4f' % r2_score(y_val, yp_pred))

In [ ]:
# Plota a saída
plt.figure(figsize=(20, 15))
x = list(range(len(x_val)))
plt.scatter(x, yp_pred,  color='black')
plt.plot(x, yp_pred, color='green', linewidth=2)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
# teste modelo
regrp2_pred_teste = regrp.predict(x_test)

# Mostra o erro quadrado médio
print("Mean squared error: %.4f"
      % mean_squared_error(y_test, regrp2_pred_teste))
# Coeficiente de Determinação Rˆ2 - r ao quadrado: 1 é a previsão perfeita
print('Coeficiente de Determinação: %.4f' % r2_score(y_test, regrp2_pred_teste))

In [ ]:
# Plota a saída
plt.figure(figsize=(20, 15))
x = list(range(len(x_test)))
plt.scatter(x, y_test,  color='black')
plt.plot(x, regrp2_pred_teste, color='grey', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

## 11. Escolha um modelo dos Ensembles do Scikit-Learn.

### Utilização do modelo GradientBoosting estudado em aula

In [ ]:
# transformar em numpy.array
x_train = X_train.to_numpy()
y_train = Y_train.to_numpy()
x_val = X_val.to_numpy()
y_val = Y_val.to_numpy()
x_test = X_test.to_numpy()
y_test = Y_test.to_numpy()

In [ ]:
# índice dos atributos categóricos e numéricos
categ_atrib = [0,1,2,10,12]
num_atrib = [3,4,5,6,7,8,9,11,13,14,15]
print('numéricas', num_atrib)
print('categóricas', categ_atrib)

#normalização z-norm nas numéricas
# ordinal encoder é baseado em arvores e faz uma coluna ainda nao precisa quebrar em varias binarias
enc = ColumnTransformer([('ordinal',OrdinalEncoder(categories='auto'), categ_atrib),
                         ('z-norm',StandardScaler(), num_atrib)], verbose=True)

print(enc)
x_train = enc.fit_transform(x_train)
x_val = enc.transform(x_val)
x_test = enc.transform(x_test)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

In [ ]:
# #############################################################################
# Fit regression model
params = {'n_estimators': 5000, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = GradientBoostingRegressor(**params)

clf.fit(x_train, y_train)

ygbr_pred = clf.predict(x_val)

# Mostra o erro quadrado médio
print("Mean squared error: %.4f"
      % mean_squared_error(y_val, ygbr_pred))
# Coeficiente de Determinação Rˆ2 - r ao quadrado: 1 é a previsão perfeita
print('Coeficiente de Determinação: %.4f' % r2_score(y_val, ygbr_pred))

In [ ]:
# Plota a saída
plt.figure(figsize=(20, 15))
x = list(range(len(x_val)))
plt.scatter(x, y_val,  color='black')
plt.plot(x, ygbr_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
# teste do melhor modelo
ygbr_pred_teste = clf.predict(x_test)

# Mostra o erro quadrado médio
print("Mean squared error: %.4f"
      % mean_squared_error(y_test, ygbr_pred_teste))
# Coeficiente de Determinação Rˆ2 - r ao quadrado: 1 é a previsão perfeita
print('Coeficiente de Determinação: %.4f' % r2_score(y_test, ygbr_pred_teste))

In [ ]:
# Plota a saída
plt.figure(figsize=(20, 15))
x = list(range(len(x_test)))
plt.scatter(x, y_test,  color='black')
plt.plot(x, ygbr_pred_teste, color='grey', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
# #############################################################################
# Plot training deviance

# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, ygbr_pred_teste in enumerate(clf.staged_predict(x_test)):
    test_score[i] = clf.loss_(y_test, ygbr_pred_teste)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, clf.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

In [ ]:
# #############################################################################
# Plot feature importance
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, feature_names[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

## Conclusão

O melhor modelo deste caso foi o GradientBoosting, com coeficiente de determinação R2 de 0.68 nas validações, já a regressão linear sem a retirada dos valores teve desempenho de 0.71. O modelo GradientBoosting conseguiu um coeficiente de determinação R2 de 0.75 nos testes comparado ao de regressão linear, tendo maior quantidade de acertos. Além disso teve o menor MAE. Quanto a diminuição das features categóricas, o modelo perdeu a qualidade em acertos reduzindo de  0.75 para 0.70.

## 14. Qual é a feature que mais estava prejudicando os resultados?

As features de renda estavam fora dos padrões dos outros valores, sendo necessário realizar a normalização do conjunto de dados.